In [90]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')


In [91]:
from string import translate, maketrans

In [92]:
%store -r numerosKalaba
# numerosKalaba=[5]
nextKalaba="20-K%d"%numerosKalaba[0]
prevKalaba="20-K%d"%(numerosKalaba[0]+1)
print nextKalaba,prevKalaba

20-K1 20-K2


In [93]:
home = expanduser("~")
home = expanduser("~")
prevSerie=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%prevKalaba
nextSerie=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%nextKalaba


nomsPropres=["Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["kaleb","maira","vjolet","nabil","katiSa","nikol"]


In [94]:
with open(prevSerie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.safe_load(stream)
with open(nextSerie+"Stems.yaml", 'r') as stream:
    nextStems=yaml.safe_load(stream)

In [95]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [96]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(nextStems)

In [97]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [98]:
with open(prevSerie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.safe_load(stream)

In [99]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [100]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical,choice="rdVowel"):
    result=list(radical)
    cibles=[]
    if not choice:
        choice=rd.choice(["apophonie","rdVowel"])
    if choice=="apophonie":
        for p,l in enumerate(radical):
            if l in "iae":
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=phonology["apophonies"][result[cible]]
            return "".join(result)
        else:
            return None
    else:
        for p,l in enumerate(radical):
            if l in voyelles:
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=rd.choice(voyelles)
            return "".join(result)
        else:
            return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [101]:
apophonie("kulubu",choice="rnd")

'kulibu'

In [102]:
operations=[
    devoiserFinale,
    apophonie,apophonie,
    voiserVCV,voiserVCV,
    metathese,
    changerLongueur
           ]

In [103]:
def modifierRadical(radical):
    nOperations=rd.choice([0,1,1,1,2,2,2,3,3,3,4,5])
    while nOperations>0:
        operation=rd.choice(operations)
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    return radical

In [104]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(prevLexique[mot])
        print prevLexique[mot],racineChanges[nextLexique[mot]]
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

for mot in nextLexique:
    if not mot in prevLexique:
        ding()
        print "***"
        print mot,u"n'était pas dans le lexique précédent"
        ding()
        print "***"

Negeg Negegi
riwef riwefi
liNaT liNoT
***
villageois.HYPER, villagEOIs, villagEOIS n'est pas dans le nouveau lexique
***
mubeD mubeD
vofub fovub
jebof jebif
wudan duwun
daTeS daTaS
Navum Nivumi
sovoT sevuT
kaleb kaleb
Nawusat Nawazut
dosuno duzunu
***
loup, loups n'est pas dans le nouveau lexique
***
vudegab vudogab
tun tan
sowoT sowuD
mugit mugat
vodun vidon
Sazup Sozup
buZuw baZowa
DeNu DiNe
figoT figoTo
toviral vitural
TeniS TiniSu
geNamoS guNomaS
SoDim SoDim
nagek nugek
fuZiTes foZiDos
gumeZ gumaZ
mosoN soNomu
jiDaf Dojaf
vadoS divuS
webinir webinur
Zager gaZur
wobam wibam
TiledaS TiledaS
sodep suduba
puwak puweku
neZufik nuZufik
nijeT nijeT
veguvoj vegevuj
Dadojar Dadejar
vjolet vjolet
kom kum
***
villageois, villageoiS n'est pas dans le nouveau lexique
***
NeZor NeZoro
***
loup.HYPER, lOUP, lOUPs n'est pas dans le nouveau lexique
***
Zebaf Zebuf
zibudol zibudel
jeZugiN joZogiN
zuDun zeDun
fider foder
b p
gavijaz kavijo
taZum toZume
sadimel sadimel
Sogeges Sogeges
kuDaN kaDaN
Nuna

In [105]:
for i in range(10):
    print modifierRadical("Tonif")

Tonef
Tenif
Tinif
Tonif
noDuf
Tonifu
Tonif
Tonif
Tonuf
noDif


In [106]:
racineChanges

{'Nabul': 'DomeZ',
 'Nakol': 'fezil',
 'Seduk': 'Dojaf',
 'Sol': 'zeDun',
 'Sopig': 'Zebuf',
 'SuDew': 'gaZur',
 'Tajen': 'webinur',
 'TiNum': 'joZogiN',
 'Tinur': 'pevur',
 'Todike': 'jorut',
 'Tud': 'sep',
 'Zadet': 'juNoleg',
 'ZageN': 'duzunu',
 'Zarab': 'TiledaS',
 'ZeNuko': 'sadimel',
 'bawek': 'NabiS',
 'biliw': 'piZov',
 'bu': 'p',
 'bude': 'DiNe',
 'buliw': 'guwir',
 'burun': 'muZapuj',
 'buzili': 'bagan',
 'deDor': 'tuvej',
 'digare': 'Sogeges',
 'diker': 'Dafib',
 'diwiN': 'Duvop',
 'dolit': 'toZume',
 'faSaz': 'Dazep',
 'fareDe': 'modes',
 'faru': 'laj',
 'fut': 'zizo',
 'gamaN': 'Tuseg',
 'geZoj': 'wunozol',
 'gob': 'ToTik',
 'goguto': 'mawes',
 'gorada': 'gijeN',
 'jag': 'Zuru',
 'jarami': 'sejin',
 'jasom': 'poSob',
 'jeleni': 'kaDaN',
 'jiron': 'Togoniw',
 'jumaS': 'milape',
 'ka': 'ke',
 'kaleb': 'kaleb',
 'katiSa': 'katiSa',
 'kiraj': 'vaZiT',
 'kodow': 'NeZoro',
 'korat': 'nemiT',
 'kuZaN': 'TezuT',
 'kuzeDi': 'zibudel',
 'la': 'la',
 'ladaj': 'pejis',
 'lamot': 'duw

In [107]:
newStems=replace_keys(nextStems,racineChanges)

In [108]:
with open(nextSerie+'flowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [109]:
with open(nextSerie+"flowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [110]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  Sogeges: [terrible, terribles]
  Sozup: [différent, différents, différente, différentes]
  baZowa: [bas, basse, basses]
  foder: [jaune, jaunes]
  gadin: [maigre, maigres]
  guNomaS: [gros, grosse, grosses]
  jemibo: [profond, profonds, profonde, profondes]
  joZogiN: [courageux, courageuse, courageuses]
  kavijo: [effrayé, effrayés, effrayée, effrayées]
  kizow: [noir, noire, noirs, noires]
  milape: [grand, grande, grands, grandes]
  nemiT: [blanc, blanche, blancs, blanches]
  poleS: [furieux, furieuse, furieuses]
  rumiT: [rouge, rouges]
  ruzet: [petit, petite, petits, petites]
  sejin: [quatre]
  vojuN: [trois]
  webinur: [suivant, suivants, suivante, suivantes]
  wibam: [blessé, blessés, blessée, blessées]
DET:
  ke: [IND, un, une, des, dE]
  la: [DEF, le, la, les, l]
  p: [DEM, ce, cet, cette, ces]
NOM:
  F:
    DomeZ: [fruit, fruits]
    NoTim: [villageoise, villageoises]
    TiniSu: [louve, louves]
    benov: [lune, lunes]
    bibuS: [ombre, ombres]
    dozut: [infirmiè

In [111]:
with open(nextSerie+'newStems.yaml', 'w') as output:
    output.write(yamlText)